Get Niftis

In [ ]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
path_1 = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/Optimizer_VTAs/Clinician'
nifti_df1 = import_matrices_from_folder(path_1, file_pattern='*hemi-L.nii')

path_2 = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/Optimizer_VTAs/Optimizer'
nifti_df2 = import_matrices_from_folder(path_2, file_pattern='*hemi-L.nii')

Get Subjects

In [ ]:
# import pandas as pd
# path_3 = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/Optimizer_VTAs/master.csv'
# df3 = pd.read_csv(path_3)
# display(df3)

In [ ]:
import os
import numpy as np
import nibabel as nib
from nilearn.image import resample_img

class PairedOverlap:
    def __init__(self, path_1, path_2):
        self.p1 = path_1
        self.p2 = path_2
    
    def _handle_nan(self, img):
        return np.nan_to_num(img, nan=0, posinf=1, neginf=-1)
    
    def _threshold(self, img, t=0.1):
        return (img.get_fdata() > t).astype(int)
    
    def _get_dice_coefficient(self, img1_data, img2_data):
        intersection = np.sum(img1_data * img2_data)
        NUMERATOR = 2 * intersection
        DENOMINATOR = np.sum(img1_data) + np.sum(img2_data)
        if DENOMINATOR == 0:  # If both images are empty, return perfect overlap
            return 1.0
        D = NUMERATOR / DENOMINATOR
        if (D > 1) or (D < 0):
            raise ValueError(f"Dice coefficient is out of bounds: {D}")
        return D

    def _get_percent_overlap(self, img1_data, img2_data):
        NUMERATOR = np.sum(img1_data*img2_data)
        DENOMINATOR = np.sum(img1_data)
        V = NUMERATOR/DENOMINATOR * 100
        if (V >100) or (V < 0):
            raise ValueError(f"V is over 100, {V}")
        return V
    
    def _ensure_same_dim(self, img1, img2):
        if img1.shape != img2.shape:
            img2 = resample_img(img2,target_affine=img1.affine,target_shape=img1.shape,interpolation='nearest',force_resample=True,copy_header=True)
        return img2
    
    def _get_paired_images(self, colname):
        img1 = nib.load(os.path.join(self.p1, colname)); img2 = nib.load(os.path.join(self.p2, colname))
        img2 = self._ensure_same_dim(img1, img2)
        img1 = self._handle_nan(img1); img2 = self._handle_nan(img2)
        img1 = self._threshold(img1); img2 = self._threshold(img2)
        return img1, img2

    def get_paired_overlap(self, df1, df2, df3, subject_col):
        """df1 and df2 must have same subject names in columns"""
        df3['percent_overlap_l'] = np.zeros(shape=df3.iloc[:,0].shape)
        for subject in df3[subject_col]:
            for col1 in df1.columns:
                if str(subject) in str(col1):
                    if col1 not in df2.columns:
                        continue
                    img1, img2 = self._get_paired_images(col1)
                    perc_overlap = self._get_percent_overlap(img1,img2)
                    dice_coeffic = self._get_dice_coefficient(img1,img2)
                    idx = df3.index[df3[subject_col] == subject].to_list()
                    df3.loc[idx, 'percent_overlap_l'] = perc_overlap
                    df3.loc[idx, 'dice_coeff_l'] = dice_coeffic
        return df3

In [ ]:
poverlap = PairedOverlap(path_1,path_2)
df3 = poverlap.get_paired_overlap(nifti_df1, nifti_df2, df3, subject_col='subid')
display(df3)

In [ ]:
from scipy.stats import spearmanr
r,p = spearmanr(df3['avg_overlap'], df3['postop'])
print(r,p)

In [ ]:
import seaborn as sns
sns.scatterplot(data=df3, x='avg_overlap', y='postop')

In [ ]:
sns.boxplot(data=df3, x='avg_overlap')

In [ ]:
path_3 = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/Optimizer_VTAs/master.csv'
df3.to_csv(path_3)

In [ ]:
df3['avg_overlap'] = (df3.loc[:, 'percent_overlap_l'] + df3.loc[:,'percent_overlap_r']) / 2

In [ ]:
df3